In [ ]:
import * from utils
numbins = 15
bins = np.linspace(0,1, numbins+1)
sigma = 1500
GoGaussian = False
nF2 = 3
num_bins = 15

for rat_name, mod_name, sess_name, day_name in (('R', '1', 'OF', 'day1'),
                                                ('R', '2', 'OF', 'day1'),
                                                ('R', '3', 'OF', 'day1'),
                                                ('R', '1', 'WW', 'day1'),
                                                ('R', '2', 'WW', 'day1'),
                                                ('R', '3', 'WW', 'day1'),
                                                ('R', '1', 'OF', 'day2'),
                                                ('R', '2', 'OF', 'day2'),
                                                ('R', '3', 'OF', 'day2'),
                                                ('Q', '1', 'OF', ''),
                                                ('Q', '2', 'OF', ''),
                                                ('Q', '1', 'WW', ''),
                                                ('Q', '2', 'WW', ''),
                                                ('S', '1', 'OF', ''),
                                                ('S', '1', 'WW', ''),
                                                ):
    
    file_name =  rat_name + '_' + mod_name + '_' + sess_name
    if len(day_name)>0:
        file_name += '_' + day_name  
        
    spikes, xx, yy,__,__ = get_spikes(rat_name, mod_name, day_name, sess_name, bType = 'pure',
                                     bSmooth = False, bSpeed = True)
    T, num_neurons = np.shape( spikes)
    
    ypt_all = []
    Lvals_tor = []
    P_tor_all = np.zeros((num_neurons, num_bins**2, nF2) )
    LOOtorscores = np.zeros((num_neurons))

    filenames = glob.glob('Toroidal_topology_grid_cell_data/Results/' + file_name + '_coords*')
    coordsall = {}
    if len(filenames) == 1:
        f = np.load(filenames[0], allow_pickle = True) 
        coordsall = f['coordsall'][()]
        times = f['times']
        f.close()
    else:
        continue
    for LAM in (np.sqrt(1),np.sqrt(10),np.sqrt(100),np.sqrt(1000)):
        for n in np.arange(0, num_neurons, 1): 
            coords2 = coordsall[n].copy()
            ypt_all.append([])
            Lvals_tor.append([])
            LOOtorscores[n] = glm2d(coords2[times,:], spikes[times,n], 
                                    num_bins, True, LAM, GoGaussian, nF2)
        if GoGaussian:
            np.savez_compressed('Toroidal_topology_grid_cell_data/Results/' + file_name + '_1_G_sd' + str(int(LAM)), 
                LOOtorscores = LOOtorscores)            
        else:
            np.savez_compressed('Toroidal_topology_grid_cell_data/Results/' + file_name + '_1_P_sd' + str(int(LAM)), 
                LOOtorscores = LOOtorscores)


In [ ]:
from scipy.stats import wilcoxon

fig = plt.figure()
ax = fig.add_subplot(111)
minEV = np.inf
maxEV = -np.inf

Explained_deviance_Toroidal_OF = []
Explained_deviance_Spatial_OF = []

for rat_name, mod_name, sess_name, day_name in (('R', '1', 'OF', 'day1'),
                                                ('R', '2', 'OF', 'day1'),
                                                ('R', '3', 'OF', 'day1'),
                                                ('R', '1', 'OF', 'day2'),
                                                ('R', '2', 'OF', 'day2'),
                                                ('R', '3', 'OF', 'day2'),
                                                ('Q', '1', 'OF', ''),
                                                ('Q', '2', 'OF', ''),
                                                ('S', '1', 'OF', ''),
                                               ):            
    file_name = rat_name + '_' + mod_name + '_' + sess_name
    torscores = []
    spacescores = []
    torscoresmean = []
    spacescoresmean = []
    for LAM in (np.sqrt(1),np.sqrt(10),np.sqrt(100),np.sqrt(1000)):
        f = np.load('Toroidal_topology_grid_cell_data/Results/' + file_name + '_info.npz', allow_pickle = True)
        torscores.append(f['I_torus'])
        spacescores.append(f['I_space'])
        torscoresmean.append(np.mean(f['I_torus']))
        spacescores.append(np.mean(f['I_space']))
        f.close()

    torscores = torscores[np.argmax(torscoresmean)]
    spacescores = spacescores[np.argmax(spacescoresmean)]
    maxEV =  max(maxEV, max(np.max(torscores), np.max(spacescores)))
    minEV =  min(minEV, min(np.min(torscores), np.min(spacescores)))
    ax.scatter(spacescores, torscores, s = 10, c = colors_envs[file_name])
    p  = wilcoxon(torscores-spacescores, alternative='greater')
    print(file_name)
    print(' n: ' + str(len(torscores)) )
    print(' p: ' + str(p) )
    print('')

    Explained_deviance_Toroidal_OF.extend([np.mean(torscores)])
    Explained_deviance_Toroidal_OF.extend([np.std(torscores)/np.sqrt(len(torscores))])
    Explained_deviance_Spatial_OF.extend([np.mean(spacescores)])
    Explained_deviance_Spatial_OF.extend([np.std(spacescores)/np.sqrt(len(spacescores))])
ax.plot([minEV, maxEV], [minEV, maxEV], c='k', zorder = -5)
ax.set_xlim([minEV-(maxEV-minEV)*0.05,maxEV+(maxEV-minEV)*0.05])
ax.set_ylim([minEV-(maxEV-minEV)*0.05,maxEV+(maxEV-minEV)*0.05])
ax.set_aspect('equal', 'box')
ax.xaxis.set_tick_params(width=1, length =5)
ax.yaxis.set_tick_params(width=1, length =5)

plot_stats(Explained_deviance_Toroidal_OF, Explained_deviance_Spatial_OF,  
    Explained_deviance_OF_lbls, 'info', 'OF')


In [ ]:
from scipy.stats import wilcoxon

fig = plt.figure()
ax = fig.add_subplot(111)
minEV = np.inf
maxEV = -np.inf

Explained_deviance_ToroidalC = []
Explained_deviance_SpatialC = []

for rat_name, mod_name, sess_name, day_name in (('R', '1', 'WW', 'day1'),
                                                ('R', '2', 'WW', 'day1'),
                                                ('R', '3', 'WW', 'day1'),
                                                ('Q', '1', 'WW', ''),
                                                ('Q', '2', 'WW', ''),
                                                ('S', '1', 'WW', ''),
                                               ):            
            file_name = rat_name + '_' + mod_name + '_' + sess_name
            torscores = []
            spacescores = []
            torscoresmean = []
            spacescoresmean = []
            for LAM in (np.sqrt(1),np.sqrt(10),np.sqrt(100),np.sqrt(1000)):
                f = np.load('Toroidal_topology_grid_cell_data/Results/' + file_name + '_info.npz', allow_pickle = True)
                torscores = f['I_torus']
                spacescores = f['I_space']
                torscoresmean.append(np.mean(f['I_torus']))
                spacescores.append(np.mean(f['I_space']))
                f.close()
            torscores = torscores[np.argmax(torscoresmean)]
            spacescores = spacescores[np.argmax(spacescoresmean)]

            maxEV =  max(maxEV, max(np.max(torscores), np.max(spacescores)))
            minEV =  min(minEV, min(np.min(torscores), np.min(spacescores)))
            ax.scatter(spacescores, torscores, s = 10, c = colors_envs[file_name])
            p  = wilcoxon(torscores-spacescores, alternative='greater')
            print(file_name)
            print(' n: ' + str(len(torscores)) )
            print(' p: ' + str(p) )
            print('')

            Explained_deviance_ToroidalC.extend([np.mean(torscores)])
            Explained_deviance_ToroidalC.extend([np.std(torscores)/np.sqrt(len(torscores))])
            Explained_deviance_SpatialC.extend([np.mean(spacescores)])
            Explained_deviance_SpatialC.extend([np.std(spacescores)/np.sqrt(len(spacescores))])
ax.plot([minEV, maxEV], [minEV, maxEV], c='k', zorder = -5)
ax.set_xlim([minEV-(maxEV-minEV)*0.05,maxEV+(maxEV-minEV)*0.05])
ax.set_ylim([minEV-(maxEV-minEV)*0.05,maxEV+(maxEV-minEV)*0.05])
ax.set_aspect('equal', 'box')
ax.xaxis.set_tick_params(width=1, length =5)
ax.yaxis.set_tick_params(width=1, length =5)

plot_stats(Explained_deviance_ToroidalC, Explained_deviance_SpatialC,  
    Explained_devianceC_lbls, 'ED', 'WW')
